In [1]:
# Choose the group and get the id of this group
import vk
access_token = '60ef112660ef112660ef112689608209fe660ef60ef11263d62ac435ca5b7d42d7fd64c'
session = vk.Session(access_token)
vk_api = vk.API(session, v='5.89')
group = 'pomogashkivtpu'
m = vk_api.groups.getMembers(group_id = group)

In [2]:
# Get the community members
import time
members = m['items']
count_members = m['count'] // 1000
for i in range(1, count_members+1):
    members1000 = vk_api.groups.getMembers(group_id=group, offset=i*1000)['items'] 
    members +=members1000 
    time.sleep(0.5)
    
print(len(members))

3231


In [3]:
# Function for looking up at user data
def extract_userdata(user):
    status = vk_api.users.get(user_id = user).pop() 
    return status

In [4]:
#Data preprocessing
number_private = 0 # closed profile counter
number_deactivated = 0 # counter of deleted or banned users
for m_key in members:
    status = extract_userdata(m_key)
    for status_key in status.keys(): # take the privacy field we need
        if status_key == 'deactivated': # check if the profile is deleted or blocked
            number_deactivated += 1
            members.remove(m_key)
            continue
        if status_key == 'is_closed': # check if the profile is closed or not
            if status[status_key] == True:
                number_private += 1
                members.remove(m_key)
                continue
            
print(number_private)
print(number_deactivated)
print(len(members))
print('Verification amount:'+str(number_private+number_deactivated+len(members)))

333
327
2571
Verification amount:3231


In [5]:
#Function for list for community member number m
def extract_groups(user):
    listOfSubGr = vk_api.users.getSubscriptions(user_id = user)['groups']
    return listOfSubGr

In [6]:
# Data processing
from collections import defaultdict
number_errors = 0 # counter of unsuccessfully analyzed users for subscriptions
number_success = 0 # counter of successfully analyzed users for subscriptions
groups = defaultdict(list) # special dictionary, where each key corresponds to a list of values
for m_key in members:
    try:
        listOfSubClear = extract_groups(m_key)['items'] # create a list with only group identifiers
        for l_key in listOfSubClear: # iterator for all subscriptions of a specific person
            groups[l_key].append(m_key) # filling in the dictionary, where the key is the group id and the values are the list of users
        number_success += 1
    except Exception:
        number_errors += 1
    
print(len(groups.keys()))
print('Number of user processing errors: ' + str(number_errors))
print('Number of successfully processed users: ' + str(number_success))

80993
Number of user processing errors: 94
Number of successfully processed users: 2477


In [ ]:
# Translate the list id of users into the number of users
dictOfMembers = {}
for g_key in groups.keys():
    dictOfMembers[g_key]=len(groups[g_key]) 

from collections import OrderedDict
d_sorted_by_value = OrderedDict(sorted(dictOfMembers.items(), key=lambda x: x[1]))

In [ ]:
# Display results in the table
from beautifultable import BeautifulTable
table = BeautifulTable()
table.column_headers = ['Name of community', 'Quantity of subscribers']

count = 0
for key in reversed(d_sorted_by_value):
    table.append_row([vk_api.groups.getById(group_ids = key).pop()['name'], d_sorted_by_value[key]])
    count += 1
    if count == 10:
        break
    time.sleep(0.5)
    
print(table)